<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <vector>
#include <iostream>

int main() {

    size_t width = 2048;
    size_t height = 1428;
    size_t depth = 256;
    size_t channels = 3;


    size_t num_elements = width * height * depth * channels;

    try {

        std::vector<unsigned char> s(num_elements);


        std::cout << "Memory allocation successful. Vector size: " << s.size() << std::endl;


        unsigned char* data_ptr = s.data();
        for (size_t i = 0; i < num_elements; ++i) {
            data_ptr[i] = static_cast<unsigned char>(i % 256);
        }


        std::cout << "First element: " << static_cast<int>(data_ptr[0]) << std::endl;
        std::cout << "Last element: " << static_cast<int>(data_ptr[num_elements - 1]) << std::endl;
    } catch (const std::bad_alloc& e) {

        std::cerr << "Memory allocation failed: " << e.what() << std::endl;
    }

    return 0;
}

In [ ]:
#include <iostream>
#include <sys/mman.h> // mmap, munmap
#include <unistd.h>  // sysconf
#include <fcntl.h>   // O_RDWR
#include <cstring>   // memset

int main() {

    size_t width = 2048;
    size_t height = 1428;
    size_t depth = 256;
    size_t channels = 3;

    size_t num_elements = width * height * depth * channels;
    size_t size_in_bytes = num_elements * sizeof(unsigned char);


    long page_size = sysconf(_SC_PAGESIZE);

    // 필요한 메모리 크기를 페이지 크기의 배수로 맞추기
    size_t aligned_size = ((size_in_bytes + page_size - 1) / page_size) * page_size;


    unsigned char* data_ptr = static_cast<unsigned char*>(mmap(NULL, aligned_size, PROT_READ | PROT_WRITE, MAP_PRIVATE | MAP_ANONYMOUS, -1, 0));
    if (data_ptr == MAP_FAILED) {
        std::cerr << "Memory mapping failed" << std::endl;
        return 1;
    }


    memset(data_ptr, 0, aligned_size);


    for (size_t i = 0; i < num_elements; ++i) {
        data_ptr[i] = static_cast<unsigned char>(i % 256);
    }


    std::cout << "First element: " << static_cast<int>(data_ptr[0]) << std::endl;
    std::cout << "Last element: " << static_cast<int>(data_ptr[num_elements - 1]) << std::endl;


    if (munmap(data_ptr, aligned_size) == -1) {
        std::cerr << "Memory unmapping failed" << std::endl;
        return 1;
    }

    return 0;
}